In [ ]:
# import the necessary packages
import sys
#!{sys.executable} -m pip install imutils
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
import matplotlib.pyplot as plt
#!{sys.executable} -m pip install dlib
#import dlib
from time import time

In [ ]:
# load our serialized face detector from disk
protoPath = '/opt/workspace/src/models/face_detector/deploy.prototxt'
modelPath = '/opt/workspace/src/models/face_detector/res10_300x300_ssd_iter_140000.caffemodel'
embeddingPath = '/opt/workspace/src/models/face_detector/openface.nn4.small2.v1.t7'
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
embedder = cv2.dnn.readNetFromTorch(embeddingPath)

In [ ]:
one_person = '/opt/workspace/src/datasets/actor_faces/2037_cillian_murphy/cillian_murphy_1.jpg'
two_person = '/opt/workspace/src/datasets/two_people.jpeg'
original_image = cv2.imread(two_person)

In [ ]:
def cvDnnDetectFaces(image, opencv_dnn_model,multiple_faces = False, min_confidence=0.9, display=False):

    scanned_faces = 0
    embeddings = []
    image = imutils.resize(image, width=600)
    h, w, _ = image.shape
    output_image = image.copy()
    preprocessed_image = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(300, 300), mean=(104.0, 117.0, 123.0), swapRB=False, crop=False)

    opencv_dnn_model.setInput(preprocessed_image)

    scan_start = time()
    results = opencv_dnn_model.forward()    
    scan_end = time()
    scan_time = str(scan_end - scan_start)
    if display:
        print(f'Time taken to scan the image: {scan_time} seconds.')
        
    i = np.argmax(results[0, 0, :, 2])
    iteration = 0
    
    for face in results[0][0]:
        if not multiple_faces:
            if iteration != i:
                continue
        
        iteration+=1

        face_confidence = face[2]

        if face_confidence > min_confidence:
            
            # compute the (x, y)-coordinates of the bounding box for the face
            box = face[3:7] * np.array([w, h, w, h])
            (x1, y1, x2, y2) = box.astype("int")

            # extract the face ROI and grab the ROI dimensions
            face_roi = image[y1:y2, x1:x2]
            (fH, fW) = face_roi.shape[:2]

            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue
                
            scanned_faces+=1 

            # construct a blob for the face ROI, then pass the blob through our face embedding model to obtain the 128-d quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face_roi, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            
            emb_start = time()
            vec = embedder.forward()
            emb_end = time()
            emb_time = str(emb_end - emb_start)
            
            embeddings.append(vec[0])
            
            if display:
                print(f'Time taken to process face : {emb_time} seconds.')
                cv2.rectangle(output_image, pt1=(x1, y1), pt2=(x2, y2), color=(0, 255, 0), thickness=w//200)
    
    if display:
        print(f'Total faces scanned: {str(scanned_faces)} ')
        plt.figure(figsize=[20,20])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output");plt.axis('off');
        
    else:
        return embeddings

In [ ]:
embs = cvDnnDetectFaces(original_image,detector,multiple_faces = True, display = True)

In [ ]:
len(embs)

In [ ]:
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open('/opt/workspace/src/models/results/, "wb")
f.write(pickle.dumps(data))
f.close()